IMPORTS

In [1]:
import logging
from typing import Tuple, List, TypedDict, cast
from dataclasses import dataclass
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

LOGBOOK:

- Obervations:
    - There is a lot of missing values in the dataset
    - In columns sometimes there is additional irrelevant information (like the name of the column or random gibberish)

- Ideas:
    - Why don't we try to implement a solution that would predict only for rows that have all the values, rows without one certain column, rows without two certain columns, and so on?
    - Why don't we make models for each of the case and then compare them to the one that predicts for all rows?
    If we use a model that would output the probability instead of the class, we could ensamble the predictions by getting the predictions from different models, comparing the probabilities vs accuracy, getting the weights and ensambling the final model.

LOAD DATA

In [2]:
def load_data() -> Tuple[pd.DataFrame, pd.DataFrame]:
    train = pd.read_csv('./data/train.csv')
    test = pd.read_csv('./data/test.csv')
    return train, test

LOGGING

In [3]:
def setup_logger(name: str | None = None) -> logging.Logger:
    """A function to setup the logger

    Args:
        name (str | None, optional): Logger name. Defaults to None.

    Returns:
        logging.Logger: Configured logger
    """

    file_handler = logging.FileHandler("logs.log", mode="a")
    stream_handler = logging.StreamHandler()

    logging.basicConfig(
        level=logging.WARNING,
        format="%(levelname)s %(name)s %(asctime)s | %(message)s",
        datefmt="%H:%M:%S",
        handlers=[file_handler, stream_handler],
    )

    logger = logging.getLogger(name or __name__)
    logger.setLevel(logging.INFO)

    return logger

logger = setup_logger("Main")
logger.info("Logger setup complete")

INFO Main 22:45:00 | Logger setup complete


UTILS

In [9]:

@dataclass
class ColumnTypes(TypedDict):
    """
    A class to store the features and target columns of the dataset.
    
    Attributes:
    - ids: List[str]
    - labels: List[str]
    - numerical: List[str]
    - categorical: List[str]
    - targets: List[str]
    """
    ids: List[str]
    """An array of the columns that are used as the ID of the dataset."""
    labels: List[str]
    """An array of the columns that are used as the labels of the dataset. 
    Label encoding is to be used when there is a sequential correlation between the labels."""

    numerical: List[str]
    """An array of the columns that are numerical in nature."""
    categorical: List[str]
    """An array of the columns that are categorical in nature."""
    targets: List[str]
    """An array of the columns that are the target of the dataset."""

def get_columns_types() -> ColumnTypes:
    """A function that defines the types of columns in the dataset.

    Returns:
        ColumnTypes: ColumnTypes object that contains the columns of the dataset.
    """
    return ColumnTypes(
        ids=["id"],
        targets=["class"],
        labels=["gill-spacing"],
        numerical=[
            "cap-diameter",
            "stem-height",
            "stem-width",
        ],
        categorical=[
            "cap-shape", 
            "cap-surface", 
            "cap-color", 
            "does-bruise-or-bleed",
            "gill-attachment", 
            "gill-color",
            "stem-root",
            "stem-surface",
            "stem-color",
            "veil-type",
            "veil-color",
            "has-ring",
            "ring-type",
            "spore-print-color",
            "habitat",
            "season",
        ]
    )

@dataclass
class ColumnSet:
    """A class to store the information about the features of the dataset.
    """
    
    is_optional: bool
    """A boolean value to determine if the column list is optional."""
    columns: List[str]
    """An array of the columns that are to be used in the dataset."""

def get_column_sets() -> List[ColumnSet]:
    """A function that defines the possible column combinations for the dataset.
    It will create a list of ColumnSet objects that contain the possible column combinations.

    Returns:
        List[ColumnSet]: A list of ColumnSet objects that contain the possible column combinations.
    """
    return [
        ColumnSet(is_optional=True, columns=["does-bruise-or-bleed"]),
        ColumnSet(is_optional=False, columns=["stem_height", "stem_width"]),
        ColumnSet(is_optional=True, columns=["cap-diameter","cap-shape", "cap-surface", "cap-color"]),
        ColumnSet(is_optional=True, columns=["gill-spacing", "gill-attachment", "gill-color"]),
        ColumnSet(is_optional=True, columns=["stem-root", "stem-surface", "stem-color"]),
        ColumnSet(is_optional=True, columns=["veil-type", "veil-color"]),
        ColumnSet(is_optional=True, columns=["has-ring", "ring-type"]),
        ColumnSet(is_optional=True, columns=["spore-print-color", "habitat", "season"]),
    ]

def get_column_transformer(column_types: ColumnTypes, categorical_outliers_frequency_limit : float) -> Pipeline:
    """A function that creates a pipeline object that contains the transformers to be used for the columns.

    Returns:
        Pipeline: A pipeline object that contains the transformers to be used for the columns.
    """
    
    def label_encode(x: pd.DataFrame) -> pd.DataFrame:
        logger.info("Label encoding started")
        for column in x.columns:
            x[column] = pd.Series(LabelEncoder().fit_transform(x[column])).rename(column)

        logger.info("Label encoding complete")
        return x
    
    def clean_categorical_columns(x: pd.DataFrame) -> pd.DataFrame:
        logger.info("Cleaning categorical columns started")
        logger.info(f"Outliers frequency limit is {categorical_outliers_frequency_limit}")
        for column in x.columns:
            if column in [*column_types["labels"], *column_types["categorical"]]: 
                
                value_counts = x[column].value_counts().to_frame()
                sum_value_counts = value_counts["count"].sum()

                outliers = cast(pd.DataFrame, value_counts[value_counts["count"] < sum_value_counts * categorical_outliers_frequency_limit]).index.to_list()
                logger.info(f"Outliers for column '{column}' are {outliers}")
                x[column] = pd.Series(x[column].apply(lambda el : el if el not in outliers else "gibberish")).rename(column)


                cleaned_series = x[column]
                x[column] = cleaned_series

        logger.info("Cleaning categorical columns complete")
        return x
    
    def clean_numerical_columns(x: pd.DataFrame) -> pd.DataFrame:
        logger.info("Cleaning numerical columns started")
        for column in x.columns:
            if column in column_types["numerical"]:
                n_nans = x[column].isna().sum()
                if n_nans > 0:
                    logger.info(f"Column '{column}' has {n_nans} NaNs. Filling with mean {x[column].mean()}")
                    x[column] = x[column].fillna(x[column].mean())
            
        logger.info("Cleaning numerical columns complete")
        return x


    categorical_column_cleaner = FunctionTransformer(clean_categorical_columns,)
    numerical_column_cleaner = FunctionTransformer(clean_numerical_columns)

    column_transformer = ColumnTransformer(
        transformers=[
            ("numerical", StandardScaler(), column_types["numerical"]),
            ("categorical", OneHotEncoder(drop="first"), column_types["categorical"]),
            ("labels", FunctionTransformer(label_encode), column_types["labels"]),
            ("ids", "passthrough", column_types["ids"]),
            ("targets", "passthrough", column_types["targets"]),
        ]
    )
    logger.info("Column transformer created")
    return Pipeline(
        steps=[
            ("categorical_column_cleaner", categorical_column_cleaner),
            ("numerical_column_cleaner", numerical_column_cleaner)
            ("column_transformer", column_transformer),
        ],
        verbose=True,
    )

@dataclass
class ColumnCreator:
    """A class to store the information about the columns to be created.
    """
    
    columns_sets: List[ColumnSet]
    """An array of the columns that are to be used in the dataset."""
    column_types: ColumnTypes
    """A ColumnTypes object that contains the columns of the dataset."""
    pipeline: Pipeline
    """A pipeline object that contains the transformers to be used for the columns."""

    def get_possible_columns_configs(self) -> List[List[str]]:
        """A function that returns the possible column configurations for the dataset.

        Returns:
            List[List[str]]: A list of lists that contain the possible column configurations.
        """
        optional_columns_sets = [column_set for column_set in self.columns_sets if column_set.is_optional]
        mandatory_columns_sets = [column_set for column_set in self.columns_sets if not column_set.is_optional]



        if len(optional_columns_sets) > 10:
            logger.warning(f"The number of optional columns sets is too high (more than 10) - {len(optional_columns_sets)}")
        else :
            logger.info(f"The number of optional columns sets is {len(optional_columns_sets)}")
        
        bitmap = 2 ** len(optional_columns_sets) - 1

        possible_columns_configs : List[List[str]] = []
        mandatory_columns : List[str] = [column for mandatory_set in mandatory_columns_sets for column in mandatory_set.columns]
        for i in range(1, bitmap + 1):
            columns_config : List[List[str]] = [ *mandatory_columns]
            for j in range(len(optional_columns_sets)):
                if i & (1 << j):
                    columns_config.extend(optional_columns_sets[j].columns)

            possible_columns_configs.append(columns_config)

        return possible_columns_configs
    
    def transform_columns(self, data: pd.DataFrame) -> pd.DataFrame:
        """A function that transforms the columns of the dataset.

        Args:
            data (pd.DataFrame): The dataset to be transformed.

        Returns:
            pd.DataFrame: The transformed dataset.
        """
        return self.pipeline.fit_transform(data.copy())


    

<>:153: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:153: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
/tmp/ipykernel_189422/803267626.py:153: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ("numerical_column_cleaner", numerical_column_cleaner)


DEBUG

In [5]:
train, test = load_data()

In [8]:
column_sets = get_column_sets()
column_types = get_columns_types()
column_transformer = get_column_transformer(column_types=column_types, categorical_outliers_frequency_limit=0.01)

column_creator = ColumnCreator(
    columns_sets=column_sets,
    column_types=column_types,
    pipeline=column_transformer
)
possible_columns_configs = column_creator.get_possible_columns_configs()
transformed_data = column_creator.transform_columns(train)

print(transformed_data)

INFO Main 22:46:27 | Column transformer created
INFO Main 22:46:27 | The number of optional columns sets is 7
INFO Main 22:46:27 | Cleaning categorical columns started
INFO Main 22:46:27 | Outliers frequency limit is 0.01
INFO Main 22:46:27 | Outliers for column 'cap-shape' are ['d', 'e', 'n', 'w', 't', 'g', 'y', 'r', 'a', 'u', 'z', 'l', 'i', 'k', 'h', '7 x', '3.55', 'm', '4.97', '8', '6.44', '8.3', '7.6', '17.44', '4.33', '2.82', '3.91', '6.21', '8.29', '54.78', '20.25', '3.52', '3.04', 'ring-type', '10.13', 'is p', '7.43', '0.82', '10.46', '2.77', '2.94', '12.62', '5.15', '19.04', '49.21', 'b f', '9.13', '1.66', '3.37', '7.21', '3.25', '11.12', '3 x', 'is s', '4.22', '3.6', '21.56', '6 x', '24.16', '2.85', '6.53', '19.29', '5 f', '4.3', '7.41', '2.63', '19.06']
INFO Main 22:46:29 | Outliers for column 'cap-surface' are ['n', 'f', 'p', 'u', 'a', 'm', 'o', 'r', 'x', 'c', 'b', 'z', 'season', 'ring-type', 'class', 'does None', 'has-ring', 'does t', 'is None', '5.73', '14.04', '1.42', '8.

[Pipeline] .... (step 1 of 2) Processing column_cleaner, total=  25.1s


INFO Main 22:47:00 | Label encoding started
INFO Main 22:47:01 | Label encoding complete


ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.

In [ ]:
yo = transformed_data["stem-root"].value_counts().to_frame()
outliers = cast(pd.DataFrame, yo[yo["count"] < yo["count"].sum() * 0.01])
display(yo)
yo["count"].sum()
display(outliers.index.to_list())